## import libraries

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import pandas as pd
import geopandas as gpd
import osmnx
from scipy.stats import zscore

In [ ]:
#make plot interactive
plt.ion()

# 1)  Social Indicator

### Load & clean data 

In [ ]:
# load the necessary data here 
admin = gpd.read_file("C:/Users/charl/OneDrive/Desktop/ulster/EGM722_programming/git/egm722_project/data/Stadtteile_Hamburg.shp")
stats = pd.read_csv("C:/Users/charl/OneDrive/Desktop/ulster/EGM722_programming/git/egm722_project/data/statistics_HH21.csv",encoding="utf-8", delimiter=";",decimal="." )
#admin
#print(stats.head())

In [ ]:
#rename columns
stats.columns = ['district','inhabitants', 'pop<18','%<18', 'pop>65y','%>65', 'foreignResidents', 'migration_backg', 'hh','pp_size','hh_kids', '%hh_kids','areakm2', 'pop_density','working_pop', '%working_pop', 'unemployed','%unemployed','unemployed<18','%unemployed<18', 'unemployed>65', '%unemployed>65','social_benefits', '%social_benefits','social_housing', '%social_housing']
#stats


In [ ]:
#joined population data with shapefile of admin boundaries and dropped additional columns in the admin dataset
admin_pop = pd.merge(admin,stats, left_on='stadtteil_', right_on='district', how = 'inner')

admin_pop.drop(columns=['OBJECTID','bezirk', 'stadttei_1', 'stadttei_2','pp_size', '%<18', 'hh','foreignResidents', 'migration_backg','pop<18','hh_kids', '%hh_kids', 'pop>65y', 'unemployed<18', ],axis=1, inplace=True)

#admin_pop
#fig, ax = plt.subplots(figsize=(24, 18))
#admin_pop.plot(ax=ax, alpha=1,linewidth=2, facecolor="none")

In [ ]:
# check the number of null values and drop the null values in the colums used for calculation of social status
admin_pop.isnull().mean() * 100
admin_pop.dropna(subset=['%unemployed','%social_benefits','%social_housing','%unemployed>65' ], inplace=True)

#admin_pop.dropna(inplace=True)
#admin_pop.plot()
admin_pop.head()

In [ ]:
# this line of code first subsets the dataframe so that only numeric columns relevant to the further anylsis (social status) are maintained
# code than checks standard deviation 
social_stats = admin_pop[['%unemployed','%social_benefits','%social_housing','%unemployed>65' ]]#std()

social_stats.hist()
social_stats.std()

In [ ]:
# The scipy Library in Python is used to calculate the z-score which helps to standardize the values for each column 

z_scores_admin_pop = admin_pop[['%unemployed','%social_benefits','%social_housing','%unemployed>65']].apply(zscore)

print(z_scores_admin_pop.std())

In [ ]:
#z_scores_admin_pop

### Analysis

In [ ]:
# calculates the mean of the columns and adds the results to the data frame as new column
mean_socstats = z_scores_admin_pop[['%unemployed','%social_benefits','%social_housing','%unemployed>65', ]].mean(axis=1)
z_scores_admin_pop['mean_socstats'] = mean_socstats


In [ ]:
#groups / classifies the results into equal percentiles of the respective column (equal quantiles 0 -4 (e.g. 0= lowest 20th quantile of mean / to  - 4 within highest 20th quantile))
#and saves the results to a new column
z_scores_admin_pop ['qt_soc_stats'] = pd.qcut(z_scores_admin_pop ['mean_socstats'],5,labels=[0, 1, 2, 3, 4])


In [ ]:
def add_status_column(df, zscore_mean_column):
    df['status'] = ['very low' if x == 0 else
                   'low' if x == 1 else
                   'medium' if x == 2 else
                   'high' if x == 3 else
                   'very high' for x in df[zscore_mean_column]]
    
add_status_column(z_scores_admin_pop, 'qt_soc_stats')


In [ ]:
#apply function to add status column
add_status_column(z_scores_admin_pop, 'qt_soc_stats')

In [ ]:
z_scores_admin_pop.head()

In [ ]:
#merges the social statistics computed to the admin data frame to be able to plot the data later on
admin_socstats = pd.merge(admin_pop, z_scores_admin_pop, left_index=True, right_index=True)
admin_socstats.head()

In [ ]:
#save the data frame as csv file
#admin_socstats.to_csv("./output/social_stats.csv")

### VISUALIZATION

In [ ]:
#plot output
admin_socstats.plot("status", cmap="Reds", legend=True)  

# 2) Environmental Indicators
uses noise, green areas, surface temperature? as indicator)

In [ ]:
#load the data
noise = gpd.read_file("C:/Users/charl/OneDrive/Desktop/ulster/EGM722_programming/git/egm722_project/data/Laermkarten_HH_2018-11-19.shp")
buildings = gpd.read_file("C:/Users/charl/OneDrive/Desktop/ulster/EGM722_programming/git/egm722_project/data/Gebaeude_Hamburg.shp")
green_areas = gpd.read_file("C:/Users/charl/OneDrive/Desktop/ulster/EGM722_programming/git/egm722_project/data/Oeffentliche_Gruenanlage_Hamburg.shp")

In [ ]:
# function to checks crs of different layers and reproject to epgs:Hamburg, GE to prepare for joining later on EPSG:25832
def reproject_to_local_epsg(gdf):
    # Check the CRS of the input GeoDataFrame
    print('Original CRS:', gdf.crs)

    # Reproject the GeoDataFrame to EPSG 25832
    gdf_reproj = gdf.to_crs(epsg=25832)

    # Check the CRS of the output GeoDataFrame
    print('Reprojected CRS:', gdf_reproj.crs)

    return gdf_reproj

# apply function
admin_pop_reproj = reproject_to_local_epsg(admin_pop)
noise_reproj = reproject_to_local_epsg(noise)
housing_reproj = reproject_to_local_epsg(buildings)
green_areas_reproj = reproject_to_local_epsg(green_areas)


### Green areas 

In [ ]:
#green_areas_reproj
#green_areas_reproj.columns

In [ ]:
#for green areas delete the values with code 10 - playground according to Alkis key and drop some of the unimportant columns
green_areas_reproj = green_areas_reproj[green_areas_reproj.nutzung != 10]
green_areas_reproj.drop(columns=['veroeffent', 'nutzung', 'nutz_code' ,'herrichtun', 'gesamtanla','aktualitae','idnr','belegenh_1','belegenhei','quelle_dat', 'stand'],axis=1, inplace=True)

green_areas_reproj.head()

In [ ]:
green_areas_reproj.plot()

### Calculation of statistics of green areas at district level

In [51]:
# Calculates the sum of the area  statistics for the green areas per district :
green_area_sum = green_areas_reproj.groupby(['stadtteil'])['flaeche_ha'].sum().reset_index()
green_area_sum.rename(columns={'flaeche_ha': 'green_area_total_ha'}, inplace=True)

# Calculates mean green space area per district
green_area_mean = green_areas_reproj.groupby(['stadtteil'])['flaeche_ha'].mean().reset_index()
green_area_mean.rename(columns={'flaeche_ha': 'green_area_mean_ha'}, inplace=True)

# Calculates the count of green spaces per district=  bennenung = unique name of green area
green_space_count = green_areas_reproj.groupby(['stadtteil'])['benennung'].count().reset_index()
green_space_count.rename(columns={'benennung': 'green_space_count'}, inplace=True)

# Adds the statiscts to one data frame
green_stats = pd.merge(green_area_sum, green_area_mean, on='stadtteil')
green_stats = pd.merge(green_stats, green_space_count, on='stadtteil')


In [52]:
# Calculate percentage of green space from total area of the district
green_stats['perc_green_area'] = green_areas_reproj['flaeche_ha'] / green_areas_reproj['geometry'].area * 100
green_stats.head()

,stadtteil,green_area_total_ha,green_area_mean_ha,green_space_count,perc_green_area
0,Allermöhe,41.4007,3.450058,7,0.010000
1,Alsterdorf,13.6286,0.592548,6,0.010001
2,Altengamme,0.2130,0.213000,0,0.010002
3,Altona-Altstadt,28.3002,0.496495,37,0.009993
4,Altona-Nord,8.0044,0.470847,1,0.010012


In [53]:
# Finally calculates the area per inhabitant by using the column inhabitants from the admin_pop_reproj dataframe
# ad the column to the green_stats data frame
green_stats = green_stats.merge(admin_pop_reproj[['stadtteil_', 'inhabitants']], left_on='stadtteil', right_on='stadtteil_', how='left')
green_stats.drop('stadtteil_', axis=1, inplace=True)

In [54]:
#calculate area / inhabitants 
green_stats['area_per_inhbt'] = green_stats['inhabitants'] / green_stats['green_area_total_ha']
green_stats.head()

,stadtteil,green_area_total_ha,green_area_mean_ha,green_space_count,perc_green_area,inhabitants,area_per_inhbt
0,Allermöhe,41.4007,3.450058,7,0.010000,1441.0,34.806175
1,Alsterdorf,13.6286,0.592548,6,0.010001,14975.0,1098.792246
2,Altengamme,0.2130,0.213000,0,0.010002,2319.0,10887.323944
3,Altona-Altstadt,28.3002,0.496495,37,0.009993,29151.0,1030.063392
4,Altona-Nord,8.0044,0.470847,1,0.010012,25942.0,3240.967468


In [ ]:
#check std of relevant values
#green_stats = green_stats[['area_per_inhbt','perc_green_area' ]].std()
#green_stats.std()

In [ ]:
# compute the z score of relevant columns
#z_scores_greens = green_stats[['area_per_inhbt','perc_green_area']].apply(zscore)
#z_scores_greens.head()

area_per_inhbt    1513.966291
dtype: float64

In [63]:
#groups / classifies the results into equal percentiles of the respective column (equal quantiles 0 -4 (e.g. 0= lowest 20th quantile of mean / to  - 4 within highest 20th quantile))
#and saves the results to a new column
green_stats['qt_stats_area_inhbt'] = pd.qcut(green_stats ['area_per_inhbt'],5,labels=[0, 1, 2, 3, 4])


In [65]:
# apply add column functions
add_status_column(green_stats, 'qt_stats_area_inhbt')

green_stats.head()

,stadtteil,green_area_total_ha,green_area_mean_ha,green_space_count,perc_green_area,inhabitants,area_per_inhbt,mean_greenarea,qt_green_stats,status,qt_stats_area_inhbt
0,Allermöhe,41.4007,3.450058,7,0.010000,1441.0,34.806175,NaN,0,very low,0
1,Alsterdorf,13.6286,0.592548,6,0.010001,14975.0,1098.792246,NaN,3,high,3
2,Altengamme,0.2130,0.213000,0,0.010002,2319.0,10887.323944,NaN,4,very high,4
3,Altona-Altstadt,28.3002,0.496495,37,0.009993,29151.0,1030.063392,NaN,3,high,3
4,Altona-Nord,8.0044,0.470847,1,0.010012,25942.0,3240.967468,NaN,4,very high,4


In [ ]:
#housing_reproj.columns

In [ ]:
#checks data type of column and than changes it  to integer and drops unimportant columns in the data frame

housing_reproj['gebaeudefu'] = housing_reproj['gebaeudefu'].astype(int)

#print(housing_reproj['gebaeudefu'].dtype)
housing_reproj.drop(columns=['anzahlDerU', 'lageZurErd', 'dachart', 'SHAPE_Leng'],axis=1, inplace=True)


In [ ]:
# filter out any non-residential housing from data set according to ALKIS Key (1000 and 1010 )
housing_reproj = housing_reproj[(housing_reproj['gebaeudefu'] == 1010) | (housing_reproj['gebaeudefu'] == 1000)]
#housing_reproj.head()